In [72]:
# question 1
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]") \
                    .getOrCreate()

df = spark.read.option("header",True) \
    .csv("DataSample.csv")
df.createOrReplaceTempView("data")

# clean data by collecting duplicates together
# and only keeping the first entry
sqlDF = spark.sql("SELECT * FROM data x \
                    WHERE x._ID IN \
                      (SELECT _ID FROM \
                         (SELECT _ID, ROW_NUMBER() OVER \
                            (PARTITION BY data. TimeSt, data.Country, data.Province, data.City, \
                                data.Latitude, data.Longitude ORDER BY _ID) duplicate \
                            FROM data) \
                            WHERE duplicate < 2);")

sqlDF.write.option("header",True) \
 .csv("CleanedData")

In [5]:
# question 2
import pyspark
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from geopy import distance

spark = SparkSession.builder.master("local[*]") \
                    .getOrCreate()

requests = spark.read.options(header='True',inferSchema='True',delimiter=',') \
    .csv("CleanedData.csv")
requests.createOrReplaceTempView("data")

pois = spark.read.options(header='True',inferSchema='True',delimiter=',') \
    .csv("POIList.csv")
pois.createOrReplaceTempView("interest_points")

# redefined distance function so it's usable in dataframe
def dist(a,b,x,y):
    return distance.distance((a,b),(x,y)).km
udf_dist = udf(dist, FloatType())

# ***temp edited lat & long in POIs to make this work
# create a new dataframe of size count(POIs)*requests, 
# annotated with distance between each request and each POI
distances = requests.join(pois).withColumn('Distance', udf_dist(requests.Latitude,requests.Longitude, \
                                                                pois.Latitude2,pois.Longitude2))
# find min distance per request
min_distances = distances.groupBy('_ID').min('Distance')
# annotate distances with min distances, then filter out all non-min POIs
# and delete unnecessary columns
distances = distances.join(min_distances, distances._ID == min_distances._ID).select(distances["*"],min_distances["min(Distance)"])
distances = distances.filter(distances["Distance"]==distances["min(Distance)"])
distances = distances.drop("min(Distance)", "Latitude2", "Longitude2")

distances.write.option("header",True) \
 .csv("AssignedData")

In [116]:
# question 3
# *** note we are operating with edited headers and poi2 removed
import pyspark
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]") \
                    .getOrCreate()

df = spark.read.options(header='True',inferSchema='True',delimiter=',') \
    .csv("AssignedData.csv")
df.createOrReplaceTempView("data")

# just output summary statistics *** for now
df_filtered=df.filter(df.POIID=="POI1")
df_filtered.describe('Distance').show()

df_filtered=df.filter(df.POIID=="POI3")
df_filtered.describe('Distance').show()

df_filtered=df.filter(df.POIID=="POI4")
df_filtered.describe('Distance').show()

+-------+------------------+
|summary|          distance|
+-------+------------------+
|  count|              9356|
|   mean|303.08939549834696|
| stddev|418.72973146805134|
|    min|         0.3495942|
|    max|         11541.829|
+-------+------------------+

+-------+-----------------+
|summary|         distance|
+-------+-----------------+
|  count|             9228|
|   mean|  451.77785672626|
| stddev|224.5055577157318|
|    min|        0.8095297|
|    max|        1500.4532|
+-------+-----------------+

+-------+------------------+
|summary|          distance|
+-------+------------------+
|  count|               469|
|   mean|502.84822352025566|
| stddev|1488.1157551042654|
|    min|         16.224401|
|    max|          9365.336|
+-------+------------------+

